## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

## List

In [10]:
ct("List",2,2)
exe("""
select 費目,出金額
from 家計簿
where 出金額>3000
""")

[('交際費', 5000), ('水道光熱費', 7560)]

In [11]:
ct("List",2,3)
exe("""
/*coment*/
select 費目,出金額  --coment
from 家計簿
where 出金額>3000
""")

[('交際費', 5000), ('水道光熱費', 7560)]

In [12]:
ct("List",2,7)
stmt="""
select 費目 as item, 出金額  as receive
from 家計簿 as moneybook
where 費目='給料'
"""
with engine.connect() as conn:
    df=pd.read_sql(stmt,conn)
df

,item,receive
0,給料,0


In [13]:
ct("List",2,8)
exe("""
update 家計簿
set 入金額=99999
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 99999, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 99999, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 99999, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 99999, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 99999, 7560)]

In [14]:
ct("List",2,9)
exe("""
update 家計簿
set 入金額=99999
where 日付='2018-02-03'
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 99999, 380)]

In [15]:
ct("List",2,11)
exe("""
delete from 家計簿
where 日付='2018-02-03'
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [16]:
ct("List",2,11)
exe("""
insert into 家計簿
(費目,日付,出金額)
values ('通信費','2018-02-20',6200)
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 20), '通信費', None, None, 6200)]

In [17]:
ct("List",2,12)
exe("""
insert into 家計簿
values ('2018-02-20','通信費','携帯電話料金',0,6200)
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2018, 2, 20), '通信費', '携帯電話料金', 0, 6200)]

## Q

In [18]:
ct("Q",2,3,1)
exe("""
select コード,地域,都道府県名,県庁所在地,面積
from 都道府県
""")

[('01', '北海道', '北海道', '札幌市', 83456),
 ('05', '東北', '秋田県', '秋田市', 11612),
 ('13', '関東', '東京都', '新宿区', 2187),
 ('16', '北陸', '富山県', '富山市', 4247),
 ('20', '甲信', '長野県', '長野市', 13562),
 ('21', '東海', '岐阜県', '岐阜市', 10621),
 ('34', '中国', '広島県', '広島市', 8478)]

In [19]:
ct("Q",2,3,2)
exe("""
select *
from 都道府県
""")

[('01', '北海道', '北海道', '札幌市', 83456),
 ('05', '東北', '秋田県', '秋田市', 11612),
 ('13', '関東', '東京都', '新宿区', 2187),
 ('16', '北陸', '富山県', '富山市', 4247),
 ('20', '甲信', '長野県', '長野市', 13562),
 ('21', '東海', '岐阜県', '岐阜市', 10621),
 ('34', '中国', '広島県', '広島市', 8478)]

In [20]:
ct("Q",2,3,3)
exe("""
select 地域 as area,都道府県名 as pref
from 都道府県
""")

[('北海道', '北海道'),
 ('東北', '秋田県'),
 ('関東', '東京都'),
 ('北陸', '富山県'),
 ('甲信', '長野県'),
 ('東海', '岐阜県'),
 ('中国', '広島県')]

In [21]:
ct("Q",2,4,1)
exe("""
insert into 都道府県 
(コード,地域,都道府県名,面積)
values ('26','近畿','京都',4613)
""")
exe("""
insert into 都道府県 
values ('37','四国','香川','高松',1876)
""")
exe("""
insert into 都道府県 
(都道府県名,県庁所在地)
values ('福岡','福岡')
""")
exe("select * from 都道府県")

[('01', '北海道', '北海道', '札幌市', 83456),
 ('05', '東北', '秋田県', '秋田市', 11612),
 ('13', '関東', '東京都', '新宿区', 2187),
 ('16', '北陸', '富山県', '富山市', 4247),
 ('20', '甲信', '長野県', '長野市', 13562),
 ('21', '東海', '岐阜県', '岐阜市', 10621),
 ('34', '中国', '広島県', '広島市', 8478),
 ('26', '近畿', '京都', None, 4613),
 ('37', '四国', '香川', '高松', 1876),
 (None, None, '福岡', '福岡', None)]

In [22]:
ct("Q",2,5,1)
exe("""
update 都道府県
set 県庁所在地='京都'
where コード='26'
""")
exe("""
update 都道府県
set 地域='九州',面積=4976
where コード='40'
""")
exe("select * from 都道府県")

[('01', '北海道', '北海道', '札幌', 83456),
 ('05', '東北', '秋田', '秋田', 11612),
 ('13', '関東', '東京', '新宿', 2187),
 ('16', '北陸', '富山', '富山', 4247),
 ('20', '甲信', '長野', '長野', 13562),
 ('21', '東海', '岐阜', '岐阜', 10621),
 ('34', '中国', '広島', '広島', 8478),
 ('37', '四国', '香川', '高松', 1876),
 ('26', '近畿', '京都', '京都', 4613),
 ('40', '九州', '福岡', '福岡', 4976)]

In [23]:
ct("Q",2,6)
exe("delete from 都道府県 where コード='26'")
exe("select * from 都道府県")

[('01', '北海道', '北海道', '札幌', 83456),
 ('05', '東北', '秋田', '秋田', 11612),
 ('13', '関東', '東京', '新宿', 2187),
 ('16', '北陸', '富山', '富山', 4247),
 ('20', '甲信', '長野', '長野', 13562),
 ('21', '東海', '岐阜', '岐阜', 10621),
 ('34', '中国', '広島', '広島', 8478),
 ('37', '四国', '香川', '高松', 1876),
 ('40', '九州', '福岡', '福岡', 4976)]